In [1]:
import os
import re
import json
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process

# Formatting

In this section I transform the data as released by the government into a single, coherent file. The changes are superficial and do not have any semantic effect on the data.

## Changes overview

**All strings**:
- capitalized
- removed `,`, `.`
- `&` --> `AND`

**Dollar amounts**:
- no `$` or `,`
- `-` --> `0`

**Added a `compensation` field, the sum of salary and benefits**.

## File formatting

In [2]:
def format_year(f):
    df = pd.read_csv(f'raw/{f}', encoding='latin-1')

    # columns
    df.columns = df.columns.str.strip().str.lower()
    df.rename(columns={
        'sector': 'sector',
        'ï»¿sector': 'sector', # encoding weirdness
        'last name': 'last',
        'surname': 'last',
        'first name': 'first',
        'salary paid': 'salary',
        'taxable benefits': 'benefits',
        'employer': 'employer',
        'job title': 'job',
        'position': 'job',
        'calendar year': 'year'
    }, inplace=True)

    # cleaning strings
    srepl = ',.'.maketrans({',': '', '.': '', '&': 'AND'})
    df['sector'] = df['sector'].str.strip().str.upper().str.translate(srepl)
    df['last'] = df['last'].str.strip().str.upper().str.translate(srepl)
    df['first'] = df['first'].str.strip().str.upper().str.translate(srepl)
    df['employer'] = df['employer'].str.strip().str.upper().str.translate(srepl)
    df['job'] = df['job'].str.strip().str.upper().str.translate(srepl)

    # formatting $ amounts
    nrepl = '$,'.maketrans({'$': '', ',': '', '-': '0'})
    if df.salary.dtype == 'object':
        df['salary'] = df['salary'].str.translate(nrepl).astype(float)
    if df.benefits.dtype == 'object':
        df['benefits'] = df['benefits'].str.translate(nrepl).astype(float)

    # total comp column
    df['compensation'] = df['salary'] + df['benefits']
    
    df.index.name = 'ind'
    return df

In [3]:
# raw files
files = sorted([f for f in os.listdir('raw/')])

In [4]:
# formatted year files
dfs = [format_year(f) for f in files]

## Combining

In [62]:
main = pd.concat(dfs)

In [63]:
# fixing cols
main.drop('unnamed: 8', axis=1, inplace=True)
main = main[[
    'year','last','first','salary','benefits',
    'compensation','sector','employer','job'
]]

In [64]:
main.reset_index(inplace=True, drop=True)

In [65]:
main.head()

,year,last,first,salary,benefits,compensation,sector,employer,job
0,1996,KENDALL,PERRY,194890.40,711.24,195601.64,OTHER PUBLIC SECTOR EMPLOYERS,ADDICTION RESEARCH FOUNDATION,PRESIDENT AND CEO
1,1996,REHM,JUERGEN,115603.62,403.41,116007.03,OTHER PUBLIC SECTOR EMPLOYERS,ADDICTION RESEARCH FOUNDATION,DIR SOC EVAL RESEARCH AND ACT DIR CLIN RESEARCH
2,1996,ROOM,ROBIN,149434.48,512.58,149947.06,OTHER PUBLIC SECTOR EMPLOYERS,ADDICTION RESEARCH FOUNDATION,VP RESEARCH AND COORDINATOR INTERN PROGRAMS
3,1996,KNOX,KEN W,109382.92,4921.68,114304.60,ONTARIO PUBLIC SERVICE,AGRICULTUREFOOD AND RURAL AFFAIRS,DEPUTY MINISTER
4,1996,CLIFF,BRUCE,110309.00,3157.00,113466.00,HOSPITALS,AJAX AND PICKERING GENERAL HOSPITAL,PRESIDENT AND CEO


In [66]:
# saving cleaned data to file
main.to_csv('out/main_cleaned.csv')

# Garbage Data

There is a litany of garbage data hiding in this dataset. Some, I can detect and correct.

## `year`

For some reason, some `year` fields instead are other fields like `job` or `employer`.

In [67]:
bad_years = '|'.join(main.year.unique()[24:-2].tolist())
bad_year_df = {bad_years: np.nan}
main['year'] = main['year'].replace(bad_year_df, regex=True)
main['year'] = main['year'].replace('Member (Part-time) / Membre (Ã\xa0 temps partiel)', np.nan)
main['year'] = main['year'].replace('2016', 2016)

In [68]:
main['year'].unique()

array([1996., 1997., 1998., 1999., 2000., 2001., 2002., 2003., 2004.,
       2005., 2006., 2007., 2008., 2009., 2010., 2011., 2012., 2013.,
       2014., 2015., 2016., 2017., 2018.,   nan, 2019., 2020.])

In [72]:
# there are a few na years, but all are in 2016
main['year'] = main['year'].fillna(2016)

# Inflation

In this section, I calculate an inflation-adjusted amount of each `compensation` figure. I use the annual average CPI, found [here](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1810000501), to adjust each figure to 2020 CAD.

## CPI

In [11]:
# reading in cpi data
cpi = pd.read_csv('etc/cpi.csv')
cpi = cpi[cpi['Products and product groups'] == 'All-items'][['REF_DATE', 'VALUE']]
cpi.rename(columns={'REF_DATE': 'date', 'VALUE': 'value'}, inplace=True)
cpi.set_index('date', inplace=True)

# as dict
cpi_dict = cpi.to_dict()['value']

In [12]:
cpi.head()

,value
date,
1996,88.9
1997,90.4
1998,91.3
1999,92.9
2000,95.4


## Calculating

The formula is:

$\frac{137 - i}{i}$

Where:
- $i$ is the given year's annual average CPI
- $137$ is the annual average CPI for 2020, the year to which all figures are adjusted

In [73]:
def inflation(y,v):
    
    index = cpi_dict[y]
    rate = (137 - index) / index 
    amount = round(v * (1 + rate), 2)
    
    return amount

In [74]:
# cols needed
c = main[['year','compensation']]

In [75]:
# calling function
inf = c.apply(lambda x: inflation(x.year, x.compensation), axis=1)

In [76]:
# comparing inflation to standard figures
c['inf'] = inf
c.head()

<ipython-input-76-a7756af5326d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c['inf'] = inf


,year,compensation,inf
0,1996.0,195601.64,301433.35
1,1996.0,116007.03,178773.49
2,1996.0,149947.06,231077.02
3,1996.0,114304.60,176149.95
4,1996.0,113466.00,174857.62


In [77]:
# adding to main data
main['inf'] = c['inf']
main = main[[
    'year','last','first','salary',
    'benefits','compensation','inf',
    'sector','employer','job'
]]

# Sectors

The major change in this section is that I aggregate **ALL SECONDED** dislosures into the wider "Government of Ontario - Ministries". I would disaggregate them into Education, Labour etc but the stand-alone quantities aren't large enough to be significant. Instead, they can inform our picture of government staffers.

Otherwise, sector names are aggregated together using fuzzy matching.

## Raw

In [78]:
sec = main['sector'].str.strip()

Unchanged, these are the most common values:

In [79]:
sec.value_counts()[:10]

MUNICIPALITIES AND SERVICES               407325
UNIVERSITIES                              262591
SCHOOL BOARDS                             232425
HOSPITALS AND BOARDS OF PUBLIC HEALTH     186611
HYDRO ONE AND ONTARIO POWER GENERATION    122594
GOVERNMENT OF ONTARIO - MINISTRIES        110551
CROWN AGENCIES                             74477
OTHER PUBLIC SECTOR EMPLOYERS              69525
COLLEGES                                   52017
ONTARIO POWER GENERATION                   47503
Name: sector, dtype: int64

With this many unique values:

In [80]:
len(sec.unique())

62

## Fuzzy match

Since there are so few sectors, I match against a very generous [Levenshtein score](https://en.wikipedia.org/wiki/Levenshtein_distance) of `80`, and vet the matches manually. This algorithm begins with the most common tokens and searches for matches to combine thereto; so, the most common version of the token should be the final one.

In [81]:
def match_sectors(s):
    changes = {}
    values = s.value_counts().index
    for n in values:
        # value cannot be matched already
        if n not in changes.keys():
            matches = process.extract(n, values, scorer=fuzz.partial_ratio)
            if matches:
                for m,v in matches:
                    if v >= 80 and m != n and m not in changes.values():
                        changes[m] = n
                        
    return changes

In [82]:
changes = match_sectors(sec)

### Vetting matches

These are two manual corrections:

In [83]:
changes["GOVERNMENT OF ONTARIO \u00c2\u0080\u0094 MINISTRIES"] = "GOVERNMENT OF ONTARIO - MINISTRIES"
changes["GOVERNMENT OF ONTARIO \u00c2\u0080\u0094 LEGISLATIVE ASSEMBLY AND OFFICES"] = "GOVERNMENT OF ONTARIO - LEGISLATIVE ASSEMBLY AND OFFICES"

As mentioned, all seconded disclosures are aggregated into Ministries.

In [84]:
seconded = sec[sec.str.contains('SECOND')].unique()
for st in seconded:
    changes[st] = "GOVERNMENT OF ONTARIO - MINISTRIES"

### Applying matches

In [85]:
nsec = sec.replace(changes)

In [86]:
main['sector'] = nsec

New top values and # of uniques:

In [87]:
nsec.value_counts()[:10]

MUNICIPALITIES AND SERVICES               411255
UNIVERSITIES                              262595
SCHOOL BOARDS                             232425
HOSPITALS AND BOARDS OF PUBLIC HEALTH     193504
GOVERNMENT OF ONTARIO - MINISTRIES        178565
HYDRO ONE AND ONTARIO POWER GENERATION    170097
CROWN AGENCIES                             74477
OTHER PUBLIC SECTOR EMPLOYERS              69661
COLLEGES                                   58601
ONTARIO PUBLIC SERVICE                     11324
Name: sector, dtype: int64

In [88]:
len(nsec.unique())

12

## Power companies

The most common and fifth-most common employers are `Ontario Power Generation` and `Ontario Hydro` respectively. The latter is the predecessor of the former, as of 1999. Likewise, `Hydro One` another result of the Ontario Hydro privatization has 32K entries.

Due to their similarity and subtantial size, I am deliniating OPG and all its tributaries into a single sector: `POWER`.

The current grouping is as such: while OPG is grouped with `OPG and ONTARIO HYDRO`, `HYDRO ONE` is in `CROWN AGENCIES`. This is technically true but `OPG` is also a crown corporation. 

I will make these changes after employer names have been standardized, in sec. 5.

# Employers

In [124]:
e = main['employer']

## Raw

Before any standardization, this is the state of the data.

In [125]:
e.value_counts()[:10]

ONTARIO POWER GENERATION                      137335
CITY OF TORONTO                                60606
UNIVERSITY OF TORONTO                          56310
CITY OF TORONTO - POLICE SERVICE               41861
HYDRO ONE                                      32331
ATTORNEY GENERAL                               31329
TORONTO DISTRICT SCHOOL BOARD                  29222
COMMUNITY SAFETY AND CORRECTIONAL SERVICES     27576
YORK UNIVERSITY                                24967
CITY OF OTTAWA                                 22240
Name: employer, dtype: int64

In [126]:
len(e.unique())

7135

In [127]:
sum(e.value_counts()[:25])

701989

## Simple cleaning

These small style changes will make up many of the fuzzy matches. Ultimately, I want the number of matches to be as low as possible, because I will vet them manually.

- extra whitespace
- weird unicode delimiters

In [128]:
e_repl = {
    '\u00c2\u0080\u0094': '-',
    '\u00c2\u0080\u0093': '-',
    '\u00a0': '',  
    '\u00c2\u0080\u0099': "'",
    '\u0092': "'",
    '-': ''
}

In [129]:
for k,v in e_repl.items():
    e = e.str.replace(k, v, regex=False)

In [130]:
e = e.str.strip()
e = e.str.replace(r'\s+', ' ', regex=True)

These changes have reduced uniques by 500.

In [131]:
len(e.unique())

6638

## Employer-specific standardizations

The `employer` series has a number of unique issues. These arbitrary changes are meant to fix patterns that I have personally noticed reviewing the data.

### Miscellaneous abbreviations

This series is full of common abbreviations. I standardize them as so:

In [132]:
e_abbr_repl = {
    'HLTH': 'HEALTH', 'CTR': 'CENTRE', 'CENTER': 'CENTRE',
    'CENT': 'CENTRE',
    
    'SCH': 'SCHOOL', 
    
    'BD': 'BOARD', 'BRD': 'BOARD',
    
    'HOSP': 'HOSPITAL', 'HOP': 'HOSPITAL',

    "REG'L": 'REGIONAL',    
    
    'CORP': 'CORPORATION',
    
    'SOC': 'SOCIETY',
    'ASSOC': 'ASSOCIATION',
    
    'SERV': 'SERVICES',
    'SRVCS': 'SERVICES'    
}

In [133]:
for k,v in e_abbr_repl.items():
    k = fr'\b{k}\b'
    e = e.str.replace(k, v, regex=True)

### Catholic school boards

Catholic School Boards have several common acronyms. I am standardzing them as so:

In [134]:
e_cth_repl = {
    'RCDSB': 'CATHOLIC DISTRICT SCHOOL BOARD',
    'RCSSB': 'CATHOLIC DISTRICT SCHOOL BOARD',
    'CDSB': 'CATHOLIC DISTRICT SCHOOL BOARD'
}

In [135]:
for k,v in e_cth_repl.items():
    e = e.str.replace(k, v, regex=False)

In [154]:
e[e == 'HYDRO ONE']

17662     HYDRO ONE
17663     HYDRO ONE
17664     HYDRO ONE
17665     HYDRO ONE
17666     HYDRO ONE
            ...    
691024    HYDRO ONE
691025    HYDRO ONE
691026    HYDRO ONE
691027    HYDRO ONE
691028    HYDRO ONE
Name: employer, Length: 32331, dtype: object

In [156]:
main[main['employer'] == 'ONTARIO HYDRO']

,year,last,first,salary,benefits,compensation,inf,sector,employer,job
1855,1996.0,AGOSTINO,J,104900.0,616.0,105516.0,162606.21,CROWN AGENCIES,ONTARIO HYDRO,SOLICITOR
1856,1996.0,AITCHISON,G,114690.0,6507.0,121197.0,186771.53,CROWN AGENCIES,ONTARIO HYDRO,MANAGER BUSINESS DEVELOPMENT
1857,1996.0,AL,S,105088.0,494.0,105582.0,162707.92,CROWN AGENCIES,ONTARIO HYDRO,SHIFT MTCE SUPRV (CONTROL OR MECHANICAL)
1858,1996.0,ALTMAN,M,100700.0,564.0,101264.0,156053.63,CROWN AGENCIES,ONTARIO HYDRO,MANAGER PAY SERVICES
1859,1996.0,AMANTEA,F,106427.0,527.0,106954.0,164822.25,CROWN AGENCIES,ONTARIO HYDRO,TECHNICAL SUPERVISOR
...,...,...,...,...,...,...,...,...,...,...
11551,1998.0,ZETTEL,T,108146.0,587.0,108733.0,163159.05,CROWN AGENCIES,ONTARIO HYDRO,MAINTENANCE COORDINATOR
11552,1998.0,ZIEGLER,BE,115948.0,626.0,116574.0,174924.84,CROWN AGENCIES,ONTARIO HYDRO,SHIFT SUPERINTENDENT
11553,1998.0,ZIURAITIS,J,119642.0,626.0,120268.0,180467.86,CROWN AGENCIES,ONTARIO HYDRO,SHIFT SUPERINTENDENT
11554,1998.0,ZULIANI,CL,102866.0,36510.0,139376.0,209140.33,CROWN AGENCIES,ONTARIO HYDRO,FINANCIAL TEAM LEADER


### Resulting uniques

These standardizations have reduced the uniques further by 300

In [136]:
len(e.unique())

6386

## Fuzzy matching

Like the sector matching algorithm, it begins with the most common tokens in the series.

### First pass

In [137]:
def match_employers(s):
    changes = {}
    for e in s:
        if e not in changes.keys():
            matches = process.extract(e, s, scorer=fuzz.ratio)
            if matches:
                for m,v in matches:
                    if v >= 90 and m != e and m not in changes.values():
                        changes[m] = e      
    return changes

In [139]:
changes = match_employers(e.value_counts().index)

### Saving for manual vet

In [ ]:
pd.Series(changes).to_csv('etc/emp_changes_1.csv')

### Vetted first pass

I only vetted the first 500 of this pass.

In [140]:
e5 = pd.read_csv('etc/emps 500.csv').set_index('vet_old')
e5.head()

,vet_new
vet_old,
ATTORNEYGENERAL,ATTORNEY GENERAL
CITY OF TORONTO TORONTO TRANSIT COMM,CITY OF TORONTO TORONTO TRANSIT COMMISSION
THE UNIVERSITY OF WESTERN ONTARIO,UNIVERSITY OF WESTERN ONTARIO
COMM SAFETYAND CORR SERVICES,COMM SAFETY AND CORR SERVICES
COMMUNITY SAFETY AND CORR SERVICES,COMM SAFETY AND CORR SERVICES


In [141]:
e_changes_1 = e5.to_dict()['vet_new']
e = e.replace(e_changes_1)

These changes reduce the uniques by another 200. But, these are valuable changes: they affect the most common tokens first, and so increase the concentration at the top values.

In [142]:
# new uniques
len(e.unique())

6106

In [143]:
sum(e.value_counts()[:25])

718553

### Second pass

In [144]:
# changes2 = match_employers(e.value_counts().index)

In [145]:
# pd.Series(changes2).to_csv('etc/emps_changes_2.csv')

### Vetted second pass

**Note that** some other algorithms to reduce the amount of fuzzy matches are possible. Many false positives are long names for bodies representing different regions, e.g., school boards. With a list of Ontario placenames, these could be excluded. Likewise, acronym matches between tokens like `CAS` and `CCAS` are frequent but incorrect.

In [147]:
emp_changes_2 = pd.read_csv('etc/emp_changes_2.csv')

In [148]:
emp_changes_2.head()

,vet_old,vet_new,old,new
0,NaN,NaN,REGIONAL MUNICIPALITY OF YORK POLICE SERVICES,REGIONAL MUNICIPALITY OF PEEL POLICE SERVICES
1,TORONTO METRO CATHOLIC DISTRICT SCHOOL BOARD,TORONTO CATHOLIC DISTRICT SCHOOL BOARD,TORONTO METRO CATHOLIC DISTRICT SCHOOL BOARD,TORONTO CATHOLIC DISTRICT SCHOOL BOARD
2,HALTON CATHOLIC DISTRICT SCHOOL BOARD,THE HALTON CATHOLIC DISTRICT SCHOOL BOARD,HALTON CATHOLIC DISTRICT SCHOOL BOARD,THE HALTON CATHOLIC DISTRICT SCHOOL BOARD
3,WATERLOO CATHOLIC DISTRICT SCHOOL BOARD,WATERLOO COUNTY CATHOLIC DISTRICT SCHOOL BOARD,WATERLOO CATHOLIC DISTRICT SCHOOL BOARD,WATERLOO COUNTY CATHOLIC DISTRICT SCHOOL BOARD
4,NaN,NaN,LONDON CATHOLIC DISTRICT SCHOOL BOARD,WELLINGTON CATHOLIC DISTRICT SCHOOL BOARD


In [149]:
# getting changes from DF
emp_dict_2 = emp_changes_2[['vet_old', 'vet_new']].set_index('vet_old')
emp_dict_2 = emp_dict_2[emp_dict_2['vet_new'].isna() == False]
emp_dict_2 = emp_dict_2.to_dict()['vet_new']

In [150]:
e = e.replace(emp_dict_2)

## Final state of data

In [151]:
e.value_counts()[:10]

ONTARIO POWER GENERATION                      137335
CITY OF TORONTO                                60606
UNIVERSITY OF TORONTO                          56310
CITY OF TORONTO POLICE SERVICE                 46455
HYDRO ONE                                      32331
ATTORNEY GENERAL                               31329
TORONTO DISTRICT SCHOOL BOARD                  29222
COMMUNITY SAFETY AND CORRECTIONAL SERVICES     27576
CITY OF TORONTO TORONTO TRANSIT COMMISSION     25384
YORK UNIVERSITY                                24967
Name: employer, dtype: int64

In [152]:
len(e.unique())

5405

In [153]:
sum(e.value_counts()[:25])

718572

### Adding changes

In [157]:
main['employer'] = e

## Changing sectors for power companies

As mentioned, all Ontario power companies (now standardized) are entering a new sector called `POWER`.

In [172]:
# final standardization 
# result of subsidiary name change
e_power_repl = {
    'INDEPENDENT ELECTRICITY MARKET OP': 'INDEPENDENT ELECTRICITY SYSTEM OPERATOR'
}

In [173]:
main['employer'] = main['employer'].replace(e_power_repl)

These are all the power employers I am aware of.

In [175]:
powers = [
    'ONTARIO POWER GENERATION',
    'HYDRO ONE',
    'ONTARIO HYDRO',
    'INDEPENDENT ELECTRICITY SYSTEM AUTHORITY',
    'ONTARIO ELECTRICITY PENSION SERVICES'
]

In [178]:
# secors associated with the employers
power_sectors = main[main['employer'].isin(powers)]['sector'].unique()

In [180]:
# dict to replace sector values
power_sectors_repl = {p: 'POWER' for p in power_sectors}

In [183]:
main['sector'] = main['sector'].replace(power_sectors_repl)

# Job titles

In [185]:
j = main['job']

## Raw data

Unchanged, this is the state of the data:

In [186]:
len(j.unique())

160065

In [187]:
j.value_counts()[:25]

PROFESSOR                        72246
ASSOCIATE PROFESSOR              46875
REGISTERED NURSE                 41751
CONSTABLE                        32015
POLICE CONSTABLE                 27262
SECONDARY TEACHER                21793
PRINCIPAL                        20190
SERGEANT                         18707
ELEMENTARY PRINCIPAL             16986
TEACHER                          15504
FIREFIGHTER                      12961
ASSISTANT PROFESSOR              12386
LAW ENFORCEMENT OFFICER          11510
ELEMENTARY TEACHER               10179
PRINCIPAL ELEMENTARY              9608
DETECTIVE                         9518
FIREFIGHTER OPERATION             9502
FACULTY MEMBER                    9319
PLAINCLOTHES POLICE CONSTABLE     9249
ASSISTANT CROWN ATTORNEY          8464
CAPTAIN                           8391
MANAGER                           8045
STAFF SERGEANT                    7371
VICE PRINCIPAL                    7240
NUCLEAR OPERATOR                  7087
Name: job, dtype: int64

In [189]:
j.value_counts()[:25].sum()

454159

## Basic cleaning

In [190]:
# whitespace
j = j.str.strip()
j = j.str.replace(r'\s+', ' ', regex=True)

Removing `/` and `-`, which often delimit compound, complex titles.

In [192]:
j = j.str.replace('/', '', regex=False)
j = j.str.replace('-', '', regex=False)

Reusing these French replacements from sector cleaning:

In [193]:
j_fr_repl = {
    '\u00c2\u0080\u0094': '',
    '\u00c2\u0080\u0093': '',
    '\u00a0': '', 
    '\u00c2\u0080\u0099': "'",
    '\u0092': "'",
}

In [194]:
for k,v in j_fr_repl.items():
    j = j.str.replace(k, v, regex=False)

In [196]:
# duplicative whitespace cleaning
j = j.str.strip()
j = j.str.replace(r'\s+', ' ', regex=True)

### Changes to data

10K fewer uniques.

In [197]:
len(j.unique())

150632

10K more in the top 25.

In [198]:
j.value_counts()[:25].sum()

460938

## Abbreviations

Unlike other fields, I am going to many *abbreviate longer versions* of common job designations. In other cases, I am going to abbreviate to more commonly known titles. The overall goal is to arbitrarily convert tokens of very common job titles to their most understandable version. 

In [199]:
j_title_repl = {
    r'\bASSOC\b': 'ASSOCIATE',
    r'\bDIR\b': 'DIRECTOR',
    r'\bEXEC\b': 'EXECUTIVE',
    r'\bMGR\b': 'MANAGER',
    r'\bGM\b': 'GENERAL MANAGER',
    r'\bVP\b': 'VICE PRESIDENT',
    'VICEPRESIDENT': 'VICE PRESIDENT',
    'VICEPRINCIPAL': 'VICE PRINCIPAL',
    
    'CHIEF EXECUTIVE OFFICER': 'CEO',
    'CHIEF FINANCIAL OFFICER': 'CFO'
}

But, I am re-using the abbreviations dict from cleaning the employers column to expand commonly abbreviated words like `centre`.

In [200]:
j_abbr_repl = {
    r'\bHLTH\b': 'HEALTH', r'\bCTR\b': 'CENTRE', r'\bCENTER\b': 'CENTRE',
    r'\bCENT\b': 'CENTRE',
    
    r'\bSCH\b': 'SCHOOL', 
    
    r'\bBD\b': 'BOARD', r'\bBRD\b': 'BOARD',
    
    r'\bHOSP\b': 'HOSPITAL', r'\bHOP\b': 'HOSPITAL',

    r"\bREG'L\b": 'REGIONAL',    
    
    r'\bCORP\b': 'CORPORATION',
    
    r'\bSOC\b': 'SOCIETY',
    
    r'\bSERV\b': 'SERVICES',
    r'\bSRVCS\b': 'SERVICES'    
}

I am also making these arbitrary changes to the most common job titles, to avoid making them with fuzzy matching. That said, most of these would score `s=100` with `fuzz.partial_ratio`.

In [201]:
j_common_repl = {
    'POLICE CONSTABLE': 'CONSTABLE',
    'POLICE SERGEANT': 'SERGEANT',
    
    'PRINCIPAL ELEMENTARY': 'PRINCIPAL',
    'ELEMENTARY PRINCIPAL': 'PRINCIPAL',
    'SECONDARY PRINCIPAL': 'PRINCIPAL',
    'PRINCIPAL SECONDARY': 'PRINCIPAL',
    
    'VICE PRINCIPAL ELEMENTARY': 'VICE PRINCIPAL',
    'ELEMENTARY VICE PRINCIPAL': 'VICE PRINCIPAL',
    'SECONDARY VICE PRINCIPAL': 'VICE PRINCIPAL',
    'VICE PRINCIPAL SECONDARY': 'VICE PRINCIPAL',
    
    'ELEMENTARY TEACHER': 'TEACHER',
    'TEACHER ELEMENTARY': 'TEACHER',
    'TEACHER SECONDARY': 'TEACHER',
    'SECONDARY TEACHER': 'TEACHER',
    "TEACHERSECONDARY": "TEACHER",
    'TEACHERELEMENTARY': 'TEACHER',
    
    'FULL PROFESSOR': 'PROFESSOR',
    
    'NURSE REGISTERED': 'REGISTERED NURSE',
    
    # FRENCH
    "ENSEIGNANTE": "TEACHER",
    'REGISTERED NURSEINFIRMIÃ¨RE AUTORISÃ©E': 'REGISTERED NURSE',
    'REGISTERED NURSE INFIRMIÃ¨RE AUTORISÃ©E': 'REGISTERED NURSE',
    'PROFESSEUR(E) AGRÃ©GÃ©(E) ASSOCIATE PROFESSOR': 'ASSOCIATE PROFESSOR',
    'PROFESSEUR(E) TITULAIRE FULL PROFESSOR': 'PROFESSOR',
    'PROFESSEUR': 'PROFESSOR',
    "LAW ENFORCEMENT OFFICER AGENT D'EXÃ©CUTION DE LA LOI": 'LAW ENFORCEMENT OFFICER'
    "LAW ENFORCEMENT OFFICER AGENTE D'EXÃ©CUTION DE LA LOI" "LAW ENFORCEMENT OFFICER",
    "ASSISTANT CROWN ATTORNEY PROCUREUR ADJOINT DE LA COURONNE": "ASSISTANT CROWN ATTORNEY",
    "ASSISTANT CROWN ATTORNEY PROCUREURE ADJOINTE DE LA COURONNE": "ASSISTANT CROWN ATTORNEY"
}

Some of these aggregations obscure smaller groups, like primary vs secondary principals. These distinctions may be useful, but are likely bad samples as virtually all of the `principal` records are either elementary or secondary school principles, but are not designated as such; and the records can still be disaggregated by sorting by sector and employer.

### Applying changes

In [202]:
for k,v in j_title_repl.items():
    j = j.str.replace(k, v, regex=True)

In [203]:
for k,v in j_abbr_repl.items():
    j = j.str.replace(k, v, regex=True)

In [204]:
j = j.replace(j_common_repl)

### Changes to data

3K fewer uniques.

In [207]:
len(j.unique())

147271

105K more in the top 25; **very substantial** change.

In [206]:
sum(j.value_counts()[:25])

565224

## Fuzzy matching

There are far too many job tokens to fully match. For the sake of my time, I am matching against the first 1000 tokens, and manually vetting.

In [210]:
def match_jobs(j, n):
    changes = {}
    for i,e in enumerate(j):
        if e not in changes.keys():
            matches = process.extract(e, j[i+1:], scorer=fuzz.token_sort_ratio)
            if matches:
                for m,v in matches:
                    if v >= 90 and m != e and m not in changes.values():
                        changes[m] = e
        if i == n:
            return changes

In [211]:
changes = match_jobs(j.value_counts().index, 1000)

KeyboardInterrupt: 

### Applying 700 rows of changes

I manually vetted 700 rows of suggested changes. The further down the list of ~2000, the less impactful they will be. 

The other way to address close misses, if I haven't fixed them in this cleaning process, is to build fuzzy matching into the API for users. E.g., when passing a `search` query to one of the fields, it can return similar values not automatically matched.

In [212]:
j700 = pd.read_csv('etc/jobs 700.csv', usecols=['vet_old','vet_new'])
j700 = j700[j700['vet_old'].isna() == False]
j700.set_index('vet_old', inplace=True)

In [213]:
j700.head()

,vet_new
vet_old,
PROFESSORS,PROFESSOR
PROFESSORMD,PROFESSOR
CONSTABLEE,CONSTABLE
CONTSTABLE,CONSTABLE
REGISTERED NURSE',REGISTERED NURSE


In [214]:
# replacing values
j = j.replace(j700.to_dict()['vet_new'])

### Changes to data

Around 800 fewer uniques.

In [215]:
len(j.unique())

146662

8K more in top 25.

In [216]:
sum(j.value_counts()[:25])

573797

## Applying changes

In [217]:
main['job'] = j

# Saving final data

In [220]:
main.to_csv('main.csv')